# Bronze layer
Purpose of this layer is to be a source of truth (raw data) and also make sure my information and data is presented in a suitable data structure

Exercise is fairly straight forward and simple, so I will not use metastore, path based delta is sufficient

### Sensor data pre-processing

Sensor 1: XLSX (use binary + pandas)


Sensor 2: CSV (supported)


Sensor 4: PARQUET (supported) + PICKLE (use binary + pandas)


Sensor 5: JSON (supported, with UDF + schema)



### Imports

In [0]:
from pyspark.sql.functions import col, explode, udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, MapType

import json
import pickle
import pandas as pd

from io import BytesIO


### Schema

Can use shared explicit schema because all data sets share the same structure:
1. Created timestamp
2. Tag Key
3. Tag Value
4. Tag Quality

Also helps with union if needed

In [0]:
sensor_schema = StructType([
    StructField("created_timestamp", StringType(), True),
    StructField("tag_key", StringType(), True),
    StructField("tag_val", DoubleType(), True),
    StructField("tag_quality", StringType(), True)
])


### Paths

In [0]:
source = "abfss://source@sembcorpete.dfs.core.windows.net"
bronze = "abfss://bronze@sembcorpete.dfs.core.windows.net"


### Sensor 1 read

XLSX not supported natively by Spark, so have to use spark-excel library

NO ETL + ELT, unified source of truth

In [0]:
try:
    import openpyxl
except ImportError:
    %pip install openpyxl
    dbutils.library.restartPython()

In [0]:
#spark excel library read
bronze_sensor1_df = spark.read \
                      .format("com.crealytics.spark.excel") \
                      .option("sheetName", "Sheet1") \
                      .option("header", "true") \
                      .schema(sensor_schema) \
                      .load(f"{source}/sensor1/sensor_1.xlsx")

In [0]:
#save to bronze for silver use, schema evolution with mergeSchema
bronze_sensor1_df.write \
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true").save(f"{bronze}/sensor2/")


### Sensor 2 read
**CSV** format supported in spark, batch read file

Since we know the file exists because of Azure set-up, we can just use a simple read

NO ETL + ELT, unified source of truth

In [0]:
bronze_sensor2_df = spark.read \
                         .format('csv') \
                         .option("header", "true") \
                         .schema(sensor_schema) \
                         .load(f"{source}/sensor2/")
                        

In [0]:
#save to bronze for silver use, schema evolution with mergeSchema
bronze_sensor2_df.write \
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true").save(f"{bronze}/sensor2/")


### Sensor 4 read
**PARQUET** format supported, batch read sensor 4 parquet

**PICKLE** format not supported: Can use UDF (AI suggested, but slower processing due to UDF), so I approach this using Pandas (will be limited if file size is too big for pandas).

**Need to approach using binary file read for pickle**


 Alternatively, since we are doing batch ingestion, we can also convert file to parquet and pass the new file into azure, then use spark to process and read the parquet files!

We also save to 2 different folders in bronze container to prevent overwriting each other + cleaner design 


 NO ETL + ELT, unified source of truth

In [0]:
#sensor 4 parquet read
bronze_sensor4_df = spark.read \
                  .format('PARQUET') \
                  .option("header", "true") \
                  .schema(sensor_schema) \
                  .load(f"{source}/sensor4/sensor_4.parquet")

In [0]:
#sensor 4 parquet write
bronze_sensor4_df.write \
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true").save(f"{bronze}/sensor4/parquet/")

In [0]:
#sensor 4 pickle read
binary_df = spark.read \
                 .format("binaryFile") \
                 .load(f"{source}/sensor4/sensor_4_diff_date.pickle")

int_processing = binary_df.select("content").collect()[0][0]

pandas_df = pickle.loads(int_processing)
bronze_sensor4_dd_df = spark.createDataFrame(pandas_df, schema=sensor_schema)

In [0]:
#sensor 4 pickle write
bronze_sensor4_dd_df.write \
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true").save(f"{bronze}/sensor4/pickle/")


### Sensor 5 read
**JSON** files supported so processing is generally straight forward

However with JSON files, we need to pay attention to Schema


so we use STRUCT columns:


1. Schema enforcement


2. Improved performance

 NO ETL + ELT, unified source of truth

In [0]:
#basic udf logic from my lecture notes, can use variant json function in databricks (need to handle variant schema)
def parse_json(json_str):
    t = type(json_str)
    if json_str:
        if t == str:
            json_obj = json.loads(json_str)
        else:
            json_obj = json_str

        sensor_array = []
        for key, item in json_obj.items():
            created_timestamp = item.get("created_timestamp")
            tag_key = item.get("tag_key")
            tag_val = item.get("tag_val")
            tag_quality = item.get("tag_quality")
            sensor_array.append((created_timestamp, tag_key, tag_val, tag_quality))
        return sensor_array

json_sensor_schema = ArrayType(sensor_schema)
udf_parse_json = udf(parse_json, json_sensor_schema)


In [0]:
bronze_sensor5_df = spark.read \
                      .format("text") \
                      .option("wholetext", "true") \
                      .load(f"{source}/sensor5/") \
                      .selectExpr("CAST(value AS STRING) as raw_json") \
                      .withColumn("parsed_records", udf_parse_json(col("raw_json"))) \
                      .select(explode(col("parsed_records")).alias("record")) \
                      .select(
                          col("record.created_timestamp").alias("created_timestamp"),
                          col("record.tag_key").alias("tag_key"),
                          col("record.tag_val").alias("tag_val"),
                          col("record.tag_quality").alias("tag_quality")
                          )

In [0]:
bronze_sensor5_df.write \
                 .mode("overwrite") \
                 .format("delta") \
                 .option("mergeSchema", "true").save(f"{bronze}/sensor5")